In [1]:
import os
import openai

from langchain.retrievers import AzureCognitiveSearchRetriever
from langchain.agents import tool

from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from langchain.prompts import PromptTemplate
import getpass


from langchain.llms import OpenAI

In [17]:
api_type = "azure"
api_base_url = "https://openaidemonubiral.openai.azure.com/"
api_version = "2023-03-15-preview"
azure_api_key = "ff5c606c134e4d1dae3426a412df834a"

openai.api_type = api_type
openai.api_base = api_base_url
openai.api_version = api_version
openai.api_key = azure_api_key

os.environ["OPENAI_API_BASE"] = api_base_url
os.environ["OPENAI_API_KEY"] = azure_api_key

In [18]:
embeddings_azure = OpenAIEmbeddings(model="text-embedding-ada-002", chunk_size=1)

llm_azure_chatgpt = AzureChatOpenAI(
    deployment_name="nubiral-lab-01", 
    temperature=0, 
    openai_api_version=api_version)

In [8]:
os.environ["AZURE_COGNITIVE_SEARCH_SERVICE_NAME"] = "cognitive-searchdemo"
os.environ["AZURE_COGNITIVE_SEARCH_INDEX_NAME"] = "cosmosdb-index"
os.environ["AZURE_COGNITIVE_SEARCH_API_KEY"] = "Ndqr05h9fitPrfEh9GxNBb2vsvelw9oUgIew1VpUvYAzSeD2n9Zi"

In [19]:
retriever_cognitive_search = AzureCognitiveSearchRetriever(content_key="description")

In [20]:
retriever_cognitive_search.get_relevant_documents("soccer shoes")

[Document(page_content="Walls can't contain you. Rivals can't hold you back. On the court, you're in control. Release your full force of nature in Predator Freak. Their supportive mid-cut design and grippy rubber outsole ensure these adidas soccer shoes keep you dominating on artificial turf. 3D-printed Demonscale elements spread further across their coated mesh upper to help you improve your mastery of the ball.", metadata={'@search.score': 10.11738, 'name': 'Predator Freak.3 Turf Shoes', 'color': 'Black', 'breadcrumbs': 'Soccer/Shoes', 'rid': 'dENnWUFNZWxyNDYrQUFBQUFBQUFBQT090'}),
 Document(page_content='Every practice takes you closer to your goal. These adidas soccer socks are made with moisture-wicking yarn to help feet stay dry. A cushioned heel and toe keep you comfortable through drills and scrimmages, while arch and ankle compression provide extra support.', metadata={'@search.score': 6.831828, 'name': 'Team Speed Soccer OTC Socks', 'color': 'White', 'breadcrumbs': 'Training/A

In [3]:
from langchain.schema import BaseRetriever, Document
from typing import List


class CustomRetrieverAdidasData(BaseRetriever):
    
    def __init__(self, retriever_cognitive_search),k=10):
        
        self.retriever_cognitive_search = retriever_cognitive_search
        self.k = k
        
    def get_relevant_documents(self, query: str) -> List[Document]:
        
        documents =  self.retriever_cognitive_search.get_relevant_documents(query)
        
        if len(documents) > self.k:
            documents = documents[:self.k]
        
        for document in documents:
            
            document.page_content = self.add_sources_to_document(document)
        
        return documents
    
    async def aget_relevant_documents(self, query: str) -> List[Document]:
        raise NotImplementedError
        
    def add_sources_to_document(self, input_document):
        
        page_content_with_sources = f"Name: {input_document.metadata['name']}\nColor: {input_document.metadata['color']}\nCategories: {input_document.metadata['breadcrumbs']}\nDescription: {input_document.page_content}"
        
        return page_content_with_sources

In [4]:
custom_retriever = CustomRetrieverAdidasData(retriever_cognitive_search, 10)

NameError: name 'retriever_cognitive_search' is not defined

In [57]:
custom_retriever.get_relevant_documents("soccer shoes")

[Document(page_content="Name: Predator Freak.3 Turf Shoes\nColor: Black\nCategories: Soccer/Shoes\nDescription: Walls can't contain you. Rivals can't hold you back. On the court, you're in control. Release your full force of nature in Predator Freak. Their supportive mid-cut design and grippy rubber outsole ensure these adidas soccer shoes keep you dominating on artificial turf. 3D-printed Demonscale elements spread further across their coated mesh upper to help you improve your mastery of the ball.", metadata={'@search.score': 10.11738, 'name': 'Predator Freak.3 Turf Shoes', 'color': 'Black', 'breadcrumbs': 'Soccer/Shoes', 'rid': 'dENnWUFNZWxyNDYrQUFBQUFBQUFBQT090'}),
 Document(page_content='Name: Team Speed Soccer OTC Socks\nColor: White\nCategories: Training/Accessories\nDescription: Every practice takes you closer to your goal. These adidas soccer socks are made with moisture-wicking yarn to help feet stay dry. A cushioned heel and toe keep you comfortable through drills and scrimm

In [84]:
prompt_template = """Use the following pieces of context to answer the question at the end. 
You are a clothing recommendation system, given the question given by the user, you must recommend the products that best meet their requirements, in addition to that, try to provide reasons why the user should buy the items that you recommend.
You can only recommend the products that are provided below, if none of the products meet the customer's requirements, kindly indicate that you do not have something that meets their requirements. Don't try to make up an answer.

Products:

{context}

Question: {question}
Answer in Spanish:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [85]:
chain_type = "stuff"
chain_type_kwargs = {"prompt": PROMPT}

qa = RetrievalQA.from_chain_type(llm=llm_azure_chatgpt, chain_type=chain_type, retriever=custom_retriever, return_source_documents=True, chain_type_kwargs=chain_type_kwargs)

In [86]:
query = "puedes recomendarme unos tenis para el verano color negro?"
result = qa({"query": query})

In [87]:
result['result']

'Sí, te recomiendo los X9000L4 Shoes en color negro. Son ideales para correr distancias cortas a medias y tienen una suela de Boost que devuelve energía en cada paso. Además, su diseño inspirado en el mundo virtual con detalles semi-translúcidos y un patrón de fallas en la suela ofrece una tracción segura en las calles de la ciudad.'

In [88]:
result['source_documents']

[Document(page_content='Name: adidas Adicolor Trefoil Hoodie\nColor: Pink\nCategories: Women/Clothing\nDescription: Which color tells your story best? Rooted in our heritage of sport, Adicolor is iconic adidas style in a spectrum of shades. This hoodie puts the spotlight on the Trefoil logo. French terry fabric is everyday-level cozy.', metadata={'@search.score': 4.453742, 'name': 'adidas Adicolor Trefoil Hoodie', 'color': 'Pink', 'breadcrumbs': 'Women/Clothing', 'rid': 'dENnWUFNZWxyNDRmQVFBQUFBQUFBQT090'}),
 Document(page_content="Name: Small Crossbody Bag\nColor: Orange\nCategories: Women/Accessories\nDescription: Only need the essentials? No problem. Throw them in this small crossbody bag, and you're ready to go. Head to and from your training session in true adidas style. Bright color guarantees you won't blend into the background. \r\n\r\nThis product is made with Primegreen, a series of high-performance recycled materials.", metadata={'@search.score': 3.835175, 'name': 'Small Cro

In [89]:
query = "necesito una camisa deportiva para hombre que me sirva para climas frios"
result = qa({"query": query})
result

{'query': 'necesito una camisa deportiva para hombre que me sirva para climas frios',
 'result': 'Lo siento, no tengo algo que cumpla con tus requisitos.',
 'source_documents': [Document(page_content="Name: adidas Love Unites All Me 3-Stripes Graphic Bra\nColor: Black\nCategories: Women/Clothing\nDescription: Elevate love and celebrate Pride in this colorful adidas sports bra. It blends comfortable support with a sleek feel that's perfect for all your low-impact endeavors. Moisture-absorbing AEROREADY keeps you dry, while a mesh back panel adds ventilation. Removable pads let you customize the coverage and shape.", metadata={'@search.score': 2.6048477, 'name': 'adidas Love Unites All Me 3-Stripes Graphic Bra', 'color': 'Black', 'breadcrumbs': 'Women/Clothing', 'rid': 'dENnWUFNZWxyNDViQWdBQUFBQUFBQT090'})]}

In [91]:
query = "quiero una camisa polo para salir a la playa para hombre"
result = qa({"query": query})
result

{'query': 'quiero una camisa polo para salir a la playa para hombre',
 'result': 'Te recomiendo la camisa Mesh Broken-Stripe Polo Shirt en color azul. Es perfecta para salir a la playa ya que es cómoda y transpirable, además de estar hecha con materiales reciclados para reducir el desperdicio de plástico.',
 'source_documents': [Document(page_content="Name: Mesh Broken-Stripe Polo Shirt\nColor: Blue\nCategories: Men/Clothing\nDescription: Step up to the tee relaxed. This adidas golf polo shirt lets you focus on accurate drives and putts. Four-way stretch matches the body's natural movements. Breathable fabric keeps you comfortable spring, summer and fall. Recycled materials are one step towards reducing plastic waste.", metadata={'@search.score': 7.207326, 'name': 'Mesh Broken-Stripe Polo Shirt', 'color': 'Blue', 'breadcrumbs': 'Men/Clothing', 'rid': 'dENnWUFNZWxyNDRGQUFBQUFBQUFBQT090'}),
  Document(page_content='Name: Ultimate365 3-Stripes Polo Shirt\nColor: White\nCategories: Men/Clo